In [17]:
import requests
import random
import os

os.environ["OMP_NUM_THREADS"] = "15"
os.environ["OPENBLAS_NUM_THREADS"] = "15"
os.environ["JOBLIB_START_METHOD"] = "forkserver"

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

In [2]:
def get_posts(subreddit, time_frame, after=None, count=None):
    url = "http://reddit.com/r/" + subreddit + "/top.json?t=" + time_frame + "&limit=100"
    
    if after:
        url += "&after=" + after
    
    if count:
        url += "&count=" + str(count)
    
    params = {
        "User-Agent": "Windows/Python/1.0"
    }

#     print(url)
    response = requests.get(url, headers=params)
    response.raise_for_status()
    response_obj = response.json()
    
#     print(response_obj)
    
    return response_obj

In [3]:
def process_posts(response_obj):
    data = response_obj['data']
    posts = data['children']

    processed_posts = []
    
    for post in posts:
        post_data = post['data']
        
        score = post_data['score']
        comments = post_data['num_comments']
        
        if score < 10 or comments < 10:
            continue

        title = post_data['title']
        text = post_data['selftext']
        nsfw = post_data['over_18']

        processed_post = {
            "text": title + " " + text,
            "nsfw": nsfw
        }

        processed_posts.append(processed_post)
    
    return processed_posts

In [4]:
def make_bow(processed_posts):
    bow = set()
    
    for post in processed_posts:
        text = post["text"]
        text = text.lower()
        split_text = text.split(" ")
        
        bow.update(split_text)
    
    return bow
        
    
def get_bow_vectors(processed_posts, bow):
    feature_vectors = []
    target_labels = []
    
    for post in processed_posts:
        post_text = post["text"]
        post_label = post["nsfw"]
        post_label_encoded = 1 if post_label else 0
        
        feature_vector = []
        
        for word in bow:
            word_count = post_text.count(word)
            feature_vector.append(word_count)
        
        feature_vectors.append(feature_vector)
        target_labels.append(post_label_encoded)
    
    return feature_vectors, target_labels

### Warning: NSFW Subreddits below have NSFW names

In [5]:
# according to http://redditlist.com/all
# note: excluding r/announcements and r/blog
top_40_overall_subreddits = ["funny", "AskReddit", "gaming", "pics", "science", 
                     "worldnews", "aww", "movies", "todayilearned", "videos",
                     "Music", "IAmA", "news", "gifs", "EarthPorn", "ShowerThoughts",
                     "askscience", "Jokes", "explainlikeimfive", "books","food","LifeProTips","DIY",
                     "mildlyinteresting","Art","sports","space","gadgets","nottheonion","television",
                     "television","photoshopbattles","Documentaries","GetMotivated","listentothis",
                     "UpliftingNews","tifu","InternetIsBeautiful","history","Futurology","philosophy","OldSchoolCool"]

top_20_sfw_subreddits = ["funny", "AskReddit", "gaming", "pics", "science", 
                     "worldnews", "aww", "movies", "todayilearned", "videos",
                     "Music", "IAmA", "news", "gifs", "EarthPorn", "ShowerThoughts",
                     "askscience", "Jokes", "explainlikeimfive", "books"]

# WARNING: nsfw subreddits have nsfw names, as you'd exepct
top_20_nsfw_subreddits = ["gonewild", "nsfw", "realgirls", "nsfw_gif", "holdthemoan",
                         "imgoingtohellforthis", "bustypetite", "cumsluts", "legalteens",
                         "petitegonewild", "nsfw_gifs", "adorableporn", "girlsfinishingthejob",
                         "asiansgonewild", "rule34", "amateur", "biggerthanyouthought", "collegesluts",
                         "porninfifteenseconds", "tittydrop"]

top_40_combined_subreddits = top_20_nsfw_subreddits
top_40_combined_subreddits.extend(top_20_sfw_subreddits)

time_frame = "year"

posts_to_get_per_subreddit = 1000
all_processed_posts = []

for subreddit in top_40_combined_subreddits:
    print("Starting to fetch for subreddit: " + subreddit)
    
    retrieved_posts = 0
    after = None
    while retrieved_posts < posts_to_get_per_subreddit:
        posts = get_posts(subreddit, time_frame, after, retrieved_posts)

        processed_posts = process_posts(posts)  
        
        if len(processed_posts) == 0:
            print("warn: Got no posts that met minimum score threshold. Quitting early with only " \
                   + str(retrieved_posts) + " total posts from " + subreddit)
            break
        
        all_processed_posts.extend(processed_posts)
        retrieved_posts += len(processed_posts)

        after = posts['data']['after']
        
        if after is None:
            print("warn: Did not find 'after' value in response. Quitting early with only " \
                   + str(retrieved_posts) + " total posts from " + subreddit)
            break

Starting to fetch for subreddit: gonewild
warn: Did not find 'after' value in response. Quitting early with only 994 total posts from gonewild
Starting to fetch for subreddit: nsfw
warn: Did not find 'after' value in response. Quitting early with only 935 total posts from nsfw
Starting to fetch for subreddit: realgirls
warn: Did not find 'after' value in response. Quitting early with only 997 total posts from realgirls
Starting to fetch for subreddit: nsfw_gif
warn: Did not find 'after' value in response. Quitting early with only 963 total posts from nsfw_gif
Starting to fetch for subreddit: holdthemoan
warn: Did not find 'after' value in response. Quitting early with only 644 total posts from holdthemoan
Starting to fetch for subreddit: imgoingtohellforthis
warn: Did not find 'after' value in response. Quitting early with only 995 total posts from imgoingtohellforthis
Starting to fetch for subreddit: bustypetite
warn: Did not find 'after' value in response. Quitting early with only 87

In [6]:
# check for duplicates - TODO: maybe take set of these?
print(len([post for post in all_processed_posts if all_processed_posts.count(post) > 1]))
# all_processed_posts = [post for post in all_processed_posts if all_processed_posts.count(post) == 1]

1411


In [7]:
# check how many nsfw
nsfw_posts = [post for post in all_processed_posts if post['nsfw']]
print(len(nsfw_posts))
print(len(all_processed_posts))

17778
37315


In [8]:
random.shuffle(all_processed_posts)

train_end = int(0.8 * len(all_processed_posts))
train_set = all_processed_posts[:train_end]
test_set = all_processed_posts[train_end:]

bow = make_bow(all_processed_posts)

In [9]:
train_feature_vectors, train_target_labels = get_bow_vectors(train_set, bow)

In [10]:
test_feature_vectors, test_target_labels = get_bow_vectors(test_set, bow)

### Naive Bayes

In [13]:
# Predict using NB
nb = MultinomialNB()
nb.fit(train_feature_vectors, train_target_labels)
test_predictions = nb.predict(test_feature_vectors)

In [14]:
accuracy = accuracy_score(test_target_labels, test_predictions)
precision = precision_score(test_target_labels, test_predictions)
recall = recall_score(test_target_labels, test_predictions)
f1 = f1_score(test_target_labels, test_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F1: " + str(f1))

# Top 20 NSFW + Top 20 SFW, 1000 posts each, full count BoW with NB:
#Accuracy: 0.8222609161532279
#Precision: 0.8981581798483207
#Recall: 0.7041336353340883
#F1: 0.7893985081733059

Accuracy: 0.8222609161532279
Precision: 0.8981581798483207
Recall: 0.7041336353340883
F1: 0.7893985081733059


### Logistic regression classifier

In [12]:
logisticRegr = LogisticRegression(solver="liblinear", max_iter=500)

In [13]:
logisticRegr.fit(train_feature_vectors, train_target_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [14]:
lr_predictions = logisticRegr.predict(test_feature_vectors)

In [16]:
accuracy = accuracy_score(test_target_labels, lr_predictions)
precision = precision_score(test_target_labels, lr_predictions)
recall = recall_score(test_target_labels, lr_predictions)
f1 = f1_score(test_target_labels, lr_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F1: " + str(f1))

# Top 20 NSFW + Top 20 SFW, 1000 posts each, full count BoW with LR:
# Accuracy: 0.899370226450489
# Precision: 0.8677929739876643
# Recall: 0.9261591299370349
# F1: 0.8960265817527344

Accuracy: 0.899370226450489
Precision: 0.8677929739876643
Recall: 0.9261591299370349
F1: 0.8960265817527344


### SVM Classifier

In [20]:
svm = LinearSVC(max_iter=2000)

In [21]:
svm.fit(train_feature_vectors,train_target_labels)

/home/davetand/.local/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=2000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [22]:
svm_predictions = svm.predict(test_feature_vectors)

In [23]:
accuracy = accuracy_score(test_target_labels, svm_predictions)
precision = precision_score(test_target_labels, svm_predictions)
recall = recall_score(test_target_labels, svm_predictions)
f1 = f1_score(test_target_labels, svm_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F1: " + str(f1))

# Top 20 NSFW + Top 20 SFW, 1000 posts each, full count BoW with LinearSVM (did not converge):
# Accuracy: 0.8926705078386707
# Precision: 0.8743397275507367
# Recall: 0.9001144819690898
# F1: 0.8870399097447468

Accuracy: 0.8926705078386707
Precision: 0.8743397275507367
Recall: 0.9001144819690898
F1: 0.8870399097447468


In [11]:
print(1)

1


### MLP Classifier

In [13]:
net = MLPClassifier(activation='logistic',hidden_layer_sizes=(100, 10))
print(net)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100, 10), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)


In [14]:
# # Predict using MLP
net = MLPClassifier(activation='logistic',hidden_layer_sizes=(100, 10))
print(net)
net.fit(train_feature_vectors,train_target_labels)
net_test_predictions = net.predict(test_feature_vectors)

In [15]:
accuracy = accuracy_score(test_target_labels, test_predictions)
precision = precision_score(test_target_labels, test_predictions)
recall = recall_score(test_target_labels, test_predictions)
f1 = f1_score(test_target_labels, test_predictions)

print("Accuracy: " + str(accuracy))
print("Precision: " + str(precision))
print("Recall: " + str(recall))
print("F1: " + str(f1))

# 500 posts per 20 subreddits using full count BoW and MultinominalNB, accuracy = 0.9805352798053528
